In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import Model, GRB
import time

start_time = time.time()
# Sets
students = pd.read_csv('set_students.csv')['students'].tolist()
teachers = pd.read_csv('set_teachers.csv')['teachers'].tolist()
courses = pd.read_csv('set_courses.csv')['courses'].tolist()
timeslots = pd.read_csv('set_timeslots.csv')['timeslots'].tolist()
living_units = pd.read_csv('set_living_units.csv')['living units'].tolist()

# Parameters
H = pd.read_csv('student_total_hours.csv', sep=';').set_index('students').to_dict()['H_s']
m = pd.read_csv('student_courses_min_hours.csv', sep=';').set_index(['students', 'courses']).to_dict()['min hours']
M = pd.read_csv('student_courses_max_hours.csv', sep=';').set_index(['students', 'courses']).to_dict()['max hours']
V = pd.read_csv('student_courses.csv', sep = ';').set_index('students').groupby('students')['courses'].apply(list).to_dict()
R = (pd.read_csv('student_teacher_relation.csv', sep=';')
    .pivot(index='Student', columns='teacher', values='relation')
    .fillna(0)
    .astype(int)
    .to_dict(orient='index')
)
T_avail = pd.read_csv('living_unit_hours.csv', sep=';').set_index('living units').groupby('living units')['timeslots'].apply(set).to_dict()
u_s = pd.read_csv('student_living_unit.csv', sep=';').set_index('students').groupby('students')['living units'].apply(set).to_dict()
T_unavail_s = pd.read_csv('student_unavailable.csv', sep=';').set_index('students').groupby('students')['unavailable'].apply(set).to_dict()
T_avail_s = {}

for student, living_unit_set in u_s.items():
    # Get the single living unit from the set
    living_unit = next(iter(living_unit_set), None)
    
    if not living_unit:
        T_avail_s[student] = set()
        continue

    # Timeslots available for the living unit
    lu_timeslots = T_avail.get(living_unit, set())

    # Timeslots the student is unavailable
    unavailable = T_unavail_s.get(student, set())

    # Subtract unavailable from living unit's available
    available = lu_timeslots - unavailable

    T_avail_s[student] = available

T_unavail_l = pd.read_csv('teacher_unavailable.csv', sep=';').set_index('teachers').groupby('teachers')['timeslots'].apply(set).to_dict()
c = pd.read_csv('teacher_course_competences.csv', sep=';').set_index(['teachers', 'courses']).to_dict()['competence']
a = pd.read_csv('student_one_on_one.csv', sep=';').set_index('students').to_dict()['one-on-one']
x_prev = pd.read_csv('previous_week.csv', sep=';').set_index(['students', 'teachers', 'timeslots', 'courses']).to_dict()['assigned']# Model

students = [s for s in students if H.get(s, 0) > 0 and V.get(s)]



In [2]:
import gurobipy as gp
from gurobipy import Model, GRB

model = Model("Student Scheduling")

# Decision Variables: 
x = model.addVars(students, teachers, timeslots, courses, vtype=GRB.BINARY, name="x")
y = model.addVars(students, teachers, timeslots, vtype=GRB.BINARY, name="y") 
y_slv = model.addVars(students, teachers, courses, vtype = GRB.BINARY, name="y_slv")
z_lt = model.addVars(teachers, timeslots, vtype=GRB.BINARY, name="z_lt")
q_st = model.addVars(students, timeslots, vtype=GRB.BINARY, name="q_st")
q_stv = model.addVars(students, timeslots, courses, vtype=GRB.BINARY, name="q_svt")
q_slv = model.addVars(students, teachers, courses, vtype=GRB.BINARY, name="q_svl")
y_slv_prev = model.addVars(students, teachers, courses, vtype=GRB.BINARY, name="z_slv_prev")
Q_maxtime = model.addVar(vtype=GRB.CONTINUOUS, name="Q_maxtime")
Q_maxsubject = model.addVar(vtype=GRB.CONTINUOUS, name="Q_maxsubject")
Q_maxteacher = model.addVar(vtype=GRB.CONTINUOUS, name="Q_maxteacher")

        
# Objective: 
lambda_lt = 4/(len(teachers)*len(timeslots))
w_1 = 2/(10*sum(H[s] for s in students))
w_2 = 1/len(timeslots)
w_3 = 1/(2*len(timeslots))
w_4 = 8/(2*len(timeslots))

model.setObjective(
     w_1 *(sum(c[l, v] * x[s, l, t, v] for s in students for l in teachers for t in timeslots for v in V[s]))
    - lambda_lt * sum(z_lt[l, t] for l in teachers for t in timeslots) 
    - (w_2 * Q_maxtime + w_3 * Q_maxsubject + w_4 * Q_maxteacher)
    , 
    
    GRB.MAXIMIZE
)

# Constraints

# A student s can only get assigned to a subject in V[s]
for s in students:  
    for l in teachers:
        for v in courses:
            for t in timeslots:
                if v not in V[s]:
                    model.addConstr(x[s,l,t,v]==0)

# Teachers can only be assigned to courses they are competent for (c[l, v] > 0)

for l in teachers:
    for v in courses:
        if c[l, v] == 0:  
            for t in timeslots:
                for s in students:
                    model.addConstr(x[s, l, t, v] == 0)
    
#A student can only get assigned to available timeslots
for s in students: 
    for l in teachers:
        for v in V[s]:
            for t in timeslots:
                if t not in T_avail_s[s]:
                    model.addConstr(x[s,l,t,v]==0)
                    
# Ensure x[s,l,t,v] is zero if t is in T_unavail_l for teacher l
for s in students:
    for l in teachers:
        for t in timeslots:
            if t in T_unavail_l[l]:  
                for v in V[s]:  
                    model.addConstr(x[s, l, t, v] == 0)
                    
# 1. Each student gets at most one teacher and one course per time slot
for s in students:
    for t in T_avail_s[s]:
        model.addConstr(sum(x[s, l, t, v] for l in teachers for v in V[s]) <= 1) 
    
# 2. Total hours assigned to each student must match required hours
for s in students:
    model.addConstr(sum(x[s, l, t, v] for l in teachers for t in timeslots for v in V[s]) == H[s])

# 3. Every student is always assigned to the same teacher for a certain course
for s in students:
    for v in V[s]:
        for l in teachers:
            model.addConstr(y_slv[s,l,v] <= sum(x[s,l,t,v] for t in timeslots))
            
for s in students:
    for v in V[s]:
        for l in teachers:
            model.addConstr(y_slv[s,l,v] >= 1/len(timeslots)* sum(x[s,l,t,v] for t in timeslots))
            
for s in students:
    for v in V[s]:
        if v in ["Varia", "Zelfstudie"]:
            continue  
        model.addConstr(sum(y_slv[s,l,v] for l in teachers) == 1)
        
# 4. Each student-course must be attended within the given range
for s in students:
    for v in V[s]:
        model.addConstr(m[(s, v)] <= sum(x[s, l, t, v] for l in teachers for t in timeslots))
        model.addConstr(sum(x[s, l, t, v] for l in teachers for t in timeslots) <= M[(s, v)])
        
# 5. Relationship factor must be respected for every student-teacher combination
for s in students:
    for l in teachers:
        for t in timeslots:
            for v in V[s]:
                model.addConstr(x[s,l,t,v] <= R[s][l])

# 6. Enforce one-on-one teaching if required
for s in students:
    for l in teachers:
        for t in timeslots:
            model.addConstr(sum(x[s_prime, l, t, v_prime] for s_prime in students if s_prime != s for v_prime in V[s_prime]) 
                            <= (1-a[s]) + (1- y[s, l, t]) * 2)
            
# Soft constraint 1: the sum of x should be <= 1 for each teacher and timeslot
for l in teachers:
    for t in timeslots:
        model.addConstr(z_lt[l, t] >= sum(x[s, l, t, v] for s in students for v in V[s]) - 1)
        

# 7. Ensure y variable consistency with x
for s in students:
    for l in teachers:
        for t in timeslots:
            model.addConstr(y[s, l, t] == sum(x[s, l, t, v] for v in V[s]))        

# Extend x_prev to include all combinations with 0 for missing ones
for s in students:
    for l in teachers:
        for t in timeslots:
            for v in courses:
                if (s, l, t, v) not in x_prev:  
                    x_prev[(s, l, t, v)] = 0  
    
# Assignment Q_maxtime
for s in students:
    for t in timeslots:
        model.addConstr(q_st[s, t] >= sum(x[s, l, t, v] for l in teachers for v in V[s]) - 
                         sum(x_prev[s, l, t, v] for l in teachers for v in V[s]))
for s in students:
    for t in timeslots:
        model.addConstr(q_st[s, t] >= sum(x_prev[s, l, t, v] for l in teachers for v in V[s]) - 
                         sum(x[s, l, t, v] for l in teachers for v in V[s]))
for s in students:
    model.addConstr(Q_maxtime >= sum(q_st[s, t] for t in timeslots))

#Assignment Q_maxsubject
for s in students:
    for v in V[s]:
        for t in timeslots:
            model.addConstr(q_stv[s, t, v] >= sum(x[s, l, t, v] for l in teachers) - sum(x_prev[s, l, t, v] for l in teachers))

for s in students:
    for v in V[s]:
        for t in timeslots:
            model.addConstr(q_stv[s, t, v] >= sum(x_prev[s, l, t, v] for l in teachers) - sum(x[s, l, t, v] for l in teachers))
for s in students:
    model.addConstr(Q_maxsubject >= sum(q_stv[s, t, v] for v in V[s] for t in timeslots))

#Assignment Q_maxteacher
for s in students:
    for l in teachers:
        for v in courses:
            for t in timeslots:
                model.addConstr(y_slv_prev[s,l,v] >= x_prev[s,l,t,v])
for s in students:
    for l in teachers:
        for v in courses:
            model.addConstr(y_slv_prev[s,l,v] <= sum(x_prev[s,l,t,v] for t in timeslots))
                
for s in students:
    for l in teachers:
        for v in courses:
            model.addConstr(q_slv[s,l,v] >= y_slv[s,l,v] - y_slv_prev[s,l,v])

for s in students:
    for l in teachers:
        for v in courses:
            model.addConstr(q_slv[s,l,v] >= y_slv_prev[s,l,v] - y_slv[s,l,v])

for s in students:
    model.addConstr(Q_maxteacher >= sum(q_slv[s,l,v] for l in teachers for v in V[s]))



Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2636099
Academic license 2636099 - for non-commercial use only - registered to so___@student.kuleuven.be


In [3]:
import gurobipy as gp
from gurobipy import Model, GRB

model_soft = Model("Student Scheduling 2")

# Decision Variables: 
x_1 = model_soft.addVars(students, teachers, timeslots, courses, vtype=GRB.BINARY, name="x")
y_1 = model_soft.addVars(students, teachers, timeslots, vtype=GRB.BINARY, name="y")  
y_slv_1 = model_soft.addVars(students, teachers, courses, vtype = GRB.BINARY, name="y_slv")

z_lt_1 = model_soft.addVars(teachers, timeslots, vtype=GRB.BINARY, name="z_lt")
z_slt_1 = model_soft.addVars(students, teachers, timeslots, vtype = GRB.BINARY, name="z_slt")

q_st_1 = model_soft.addVars(students, timeslots, vtype=GRB.BINARY, name="q_st")
q_stv_1 = model_soft.addVars(students, timeslots, courses, vtype=GRB.BINARY, name="q_svt")
q_slv_1 = model_soft.addVars(students, teachers, courses, vtype=GRB.BINARY, name="q_svl")
y_slv_prev_1 = model_soft.addVars(students, teachers, courses, vtype=GRB.BINARY, name="z_slv_prev")
Q_maxtime_1 = model_soft.addVar(vtype=GRB.CONTINUOUS, name="Q_maxtime")
Q_maxsubject_1 = model_soft.addVar(vtype=GRB.CONTINUOUS, name="Q_maxsubject")
Q_maxteacher_1 = model_soft.addVar(vtype=GRB.CONTINUOUS, name="Q_maxteacher")

        
# Objective:

lambda_lt = 4/(len(teachers)*len(timeslots))
lambda_slt = 1000/(len(timeslots)*len(students))
w_1 = 2/(10*sum(H[s] for s in students))
w_2 = 1/len(timeslots)
w_3 = 1/(2*len(timeslots))
w_4 = 8/(2*len(timeslots))
 

model_soft.setObjective(
     w_1 *(sum(c[l, v] * x_1[s, l, t, v] for s in students for l in teachers for t in timeslots for v in V[s]))
    - lambda_lt * sum(z_lt_1[l, t] for l in teachers for t in timeslots) 
    - (w_2 * Q_maxtime_1 + w_3 * Q_maxsubject_1 + w_4 * Q_maxteacher_1)
    - lambda_slt * sum(z_slt_1[s,l,t] for s in students for l in teachers for t in timeslots) 
    , 
    
    GRB.MAXIMIZE
)
    

# Constraints
#A student s can only get assigned to a subject in V[s]
for s in students:  
    for l in teachers:
        for v in courses:
            for t in timeslots:
                if v not in V[s]:
                    model_soft.addConstr(x_1[s,l,t,v]==0)
    
    
#A student can only get assigned to available timeslots
for s in students: 
    for l in teachers:
        for v in V[s]:
            for t in timeslots:
                if t not in T_avail_s[s]:
                    model_soft.addConstr(x_1[s,l,t,v]==0)
                    
# Ensure x[s,l,t,v] is zero if t is in T_unavail_l for teacher l
for s in students:
    for l in teachers:
        for t in timeslots:
            if t in T_unavail_l[l]:  
                for v in V[s]:  
                    model_soft.addConstr(x_1[s, l, t, v] == 0)
                    
# 1. Each student gets at most one teacher and one course per time slot
for s in students:
    for t in T_avail_s[s]:
        model_soft.addConstr(sum(x_1[s, l, t, v] for l in teachers for v in V[s]) <= 1) 
    
# 2. Total hours assigned to each student must match required hours
for s in students:
    model_soft.addConstr(sum(x_1[s, l, t, v] for l in teachers for t in timeslots for v in V[s]) == H[s])

# 3. Every student is always assigned to the same teacher for a certain course
for s in students:
    for v in V[s]:
        for l in teachers:
            model_soft.addConstr(y_slv_1[s,l,v] <= sum(x_1[s,l,t,v] for t in timeslots))
            
for s in students:
    for v in V[s]:
        for l in teachers:
            model_soft.addConstr(y_slv_1[s,l,v] >= 1/len(timeslots)* sum(x_1[s,l,t,v] for t in timeslots))
            
for s in students:
    for v in V[s]:
        if v in ["Varia", "Zelfstudie"]:
            continue  
        model_soft.addConstr(sum(y_slv_1[s,l,v] for l in teachers) == 1)
        
# 4. Each student-course must be attended within the given range
for s in students:
    for v in V[s]:
        model_soft.addConstr(m[(s, v)] <= sum(x_1[s, l, t, v] for l in teachers for t in timeslots))
        model_soft.addConstr(sum(x_1[s, l, t, v] for l in teachers for t in timeslots) <= M[(s, v)])

# 5. Relationship factor must be respected for every student-teacher combination
for s in students:
    for l in teachers:
        for t in timeslots:
            for v in V[s]:
                model_soft.addConstr(x_1[s,l,t,v] <= R[s][l])
        

# 6. soft one-on-one constraint
for s in students:
    for l in teachers:
        for t in timeslots:
            model_soft.addConstr(z_slt_1[s,l,t]
                            >= sum(x_1[s_prime, l, t, v_prime] for s_prime in students if s_prime != s for v_prime in V[s_prime]) - ((1-a[s]) + (1- y_1[s, l, t]) * 2))

# Soft constraint 1: the sum of x should be <= 1 for each teacher and timeslot
for l in teachers:
    for t in timeslots:
        model_soft.addConstr(z_lt_1[l, t] >= sum(x_1[s, l, t, v] for s in students for v in V[s]) - 1)
        

# 7. Ensure y variable consistency with x
for s in students:
    for l in teachers:
        for t in timeslots:
            model_soft.addConstr(y_1[s, l, t] == sum(x_1[s, l, t, v] for v in V[s]))        

# Extend x_prev to include all combinations with 0 for missing ones
for s in students:
    for l in teachers:
        for t in timeslots:
            for v in courses:
                if (s, l, t, v) not in x_prev:  
                    x_prev[(s, l, t, v)] = 0  
    
# Assignment Q_maxtime
for s in students:
    for t in timeslots:
        model_soft.addConstr(q_st_1[s, t] >= sum(x_1[s, l, t, v] for l in teachers for v in V[s]) - 
                         sum(x_prev[s, l, t, v] for l in teachers for v in V[s]))
for s in students:
    for t in timeslots:
        model_soft.addConstr(q_st_1[s, t] >= sum(x_prev[s, l, t, v] for l in teachers for v in V[s]) - 
                         sum(x_1[s, l, t, v] for l in teachers for v in V[s]))
for s in students:
    model_soft.addConstr(Q_maxtime_1 >= sum(q_st_1[s, t] for t in timeslots))

#Assignment Q_maxsubject
for s in students:
    for v in V[s]:
        for t in timeslots:
            model_soft.addConstr(q_stv_1[s, t, v] >= sum(x_1[s, l, t, v] for l in teachers) - sum(x_prev[s, l, t, v] for l in teachers))

for s in students:
    for v in V[s]:
        for t in timeslots:
            model_soft.addConstr(q_stv_1[s, t, v] >= sum(x_prev[s, l, t, v] for l in teachers) - sum(x_1[s, l, t, v] for l in teachers))
for s in students:
    model_soft.addConstr(Q_maxsubject_1 >= sum(q_stv_1[s, t, v] for v in V[s] for t in timeslots))

#Assignment Q_maxteacher
for s in students:
    for l in teachers:
        for v in courses:
            for t in timeslots:
                model_soft.addConstr(y_slv_prev_1[s,l,v] >= x_prev[s,l,t,v])
for s in students:
    for l in teachers:
        for v in courses:
            model_soft.addConstr(y_slv_prev_1[s,l,v] <= sum(x_prev[s,l,t,v] for t in timeslots))
                
for s in students:
    for l in teachers:
        for v in courses:
            model_soft.addConstr(q_slv_1[s,l,v] >= y_slv_1[s,l,v] - y_slv_prev_1[s,l,v])

for s in students:
    for l in teachers:
        for v in courses:
            model_soft.addConstr(q_slv_1[s,l,v] >= y_slv_prev_1[s,l,v] - y_slv_1[s,l,v])

for s in students:
    model_soft.addConstr(Q_maxteacher_1 >= sum(q_slv_1[s,l,v] for l in teachers for v in V[s]))




In [4]:
import gurobipy as gp
from gurobipy import Model, GRB

model_soft2 = Model("Student Scheduling 3")

# Decision Variables:
x_2 = model_soft2.addVars(students, teachers, timeslots, courses, vtype=GRB.BINARY, name="x_2")
y_2 = model_soft2.addVars(students, teachers, timeslots, vtype=GRB.BINARY, name="y_2") 
y_slv_2 = model_soft2.addVars(students, teachers, courses, vtype = GRB.BINARY, name="y_slv_2")

z_lt_2 = model_soft2.addVars(teachers, timeslots, vtype=GRB.BINARY, name="z_lt_2")
z_sv_2 = model_soft2.addVars(students, courses, vtype = GRB.CONTINUOUS, name = "z_sv_2")

q_st_2 = model_soft2.addVars(students, timeslots, vtype=GRB.BINARY, name="q_st_2")
q_stv_2 = model_soft2.addVars(students, timeslots, courses, vtype=GRB.BINARY, name="q_svt_2")
q_slv_2 = model_soft2.addVars(students, teachers, courses, vtype=GRB.BINARY, name="q_svl_2")
y_slv_prev_2 = model_soft2.addVars(students, teachers, courses, vtype=GRB.BINARY, name="z_slv_prev_2")
Q_maxtime_2 = model_soft2.addVar(vtype=GRB.CONTINUOUS, name="Q_maxtime_2")
Q_maxsubject_2 = model_soft2.addVar(vtype=GRB.CONTINUOUS, name="Q_maxsubject_2")
Q_maxteacher_2 = model_soft2.addVar(vtype=GRB.CONTINUOUS, name="Q_maxteacher_2")

        
# Objective: 
lambda_lt_2 = 4/(len(teachers)*len(timeslots))
lambda_sv_2 = 1000/(len(students)*len(courses)*len(teachers))
w_1_2 = 2/(10*sum(H[s] for s in students))
w_2_2 = 1/len(timeslots)
w_3_2 = 1/(2*len(timeslots))
w_4_2 = 8/(2*len(timeslots))

model_soft2.setObjective(
     w_1_2 *(sum(c[l, v] * x_2[s, l, t, v] for s in students for l in teachers for t in timeslots for v in V[s]))
    - lambda_lt_2 * sum(z_lt_2[l, t] for l in teachers for t in timeslots) 
    - (w_2_2 * Q_maxtime_2 + w_3_2 * Q_maxsubject_2 + w_4_2 * Q_maxteacher_2)
    - lambda_sv_2*sum(z_sv_2[s,v] for s in students for v in V[s])
    , 
    
    GRB.MAXIMIZE
)

# Constraints
# A student s can only get assigned to a subject in V[s]
for s in students:  
    for l in teachers:
        for v in courses:
            for t in timeslots:
                if v not in V[s]:
                    model_soft2.addConstr(x_2[s,l,t,v]==0)
    
    
#A student can only get assigned to available timeslots
for s in students: 
    for l in teachers:
        for v in V[s]:
            for t in timeslots:
                if t not in T_avail_s[s]:
                    model_soft2.addConstr(x_2[s,l,t,v]==0)
                    
# Ensure x[s,l,t,v] is zero if t is in T_unavail_l for teacher l
for s in students:
    for l in teachers:
        for t in timeslots:
            if t in T_unavail_l[l]:  
                for v in V[s]:  
                    model_soft2.addConstr(x_2[s, l, t, v] == 0)
                    
# 1. Each student gets at most one teacher and one course per time slot
for s in students:
    for t in T_avail_s[s]:
        model_soft2.addConstr(sum(x_2[s, l, t, v] for l in teachers for v in V[s]) <= 1) 
    
# 2. Total hours assigned to each student must match required hours
for s in students:
    model_soft2.addConstr(sum(x_2[s, l, t, v] for l in teachers for t in timeslots for v in V[s]) == H[s])

# 3. Every student is always assigned to the same teacher for a certain course
for s in students:
    for v in V[s]:
        for l in teachers:
            model_soft2.addConstr(y_slv_2[s,l,v] <= sum(x_2[s,l,t,v] for t in timeslots))
            
for s in students:
    for v in V[s]:
        for l in teachers:
            model_soft2.addConstr(y_slv_2[s,l,v] >= 1/len(timeslots)* sum(x_2[s,l,t,v] for t in timeslots))
            
for s in students:
    for v in V[s]:
        if v in ["Varia", "Zelfstudie"]:
            continue  
        model_soft2.addConstr(z_sv_2[s,v] >= sum(y_slv_2[s,l,v] for l in teachers) - 1)

for s in students:
    for v in V[s]:
        if v in ["Varia", "Zelfstudie"]:
            continue 
        model_soft2.addConstr(z_sv_2[s,v] >= 1 - sum(y_slv_2[s,l,v] for l in teachers))
        
# 4. Each student-course must be attended within the given range
for s in students:
    for v in V[s]:
        model_soft2.addConstr(m[(s, v)] <= sum(x_2[s, l, t, v] for l in teachers for t in timeslots))
        model_soft2.addConstr(sum(x_2[s, l, t, v] for l in teachers for t in timeslots) <= M[(s, v)])
        
#5. Relationship factor must be respected for every student-teacher combination
for s in students:
    for l in teachers:
        for t in timeslots:
            for v in V[s]:
                model_soft2.addConstr(x_2[s,l,t,v] <= R[s][l])

# 6. Enforce one-on-one teaching if required
for s in students:
    for l in teachers:
        for t in timeslots:
            model_soft2.addConstr(sum(x_2[s_prime, l, t, v_prime] for s_prime in students if s_prime != s for v_prime in V[s_prime]) 
                            <= (1-a[s]) + (1- y_2[s, l, t]) * 2)
            
# Soft constraint 1: the sum of x should be <= 1 for each teacher and timeslot
for l in teachers:
    for t in timeslots:
        model_soft2.addConstr(z_lt_2[l, t] >= sum(x_2[s, l, t, v] for s in students for v in V[s]) - 1)
        

# 7. Ensure y variable consistency with x
for s in students:
    for l in teachers:
        for t in timeslots:
            model_soft2.addConstr(y_2[s, l, t] == sum(x_2[s, l, t, v] for v in V[s]))        

# Extend x_prev to include all combinations with 0 for missing ones
for s in students:
    for l in teachers:
        for t in timeslots:
            for v in courses:
                if (s, l, t, v) not in x_prev:  
                    x_prev[(s, l, t, v)] = 0  
    
# Assignment Q_maxtime
for s in students:
    for t in timeslots:
        model_soft2.addConstr(q_st_2[s, t] >= sum(x_2[s, l, t, v] for l in teachers for v in V[s]) - 
                         sum(x_prev[s, l, t, v] for l in teachers for v in V[s]))
for s in students:
    for t in timeslots:
        model_soft2.addConstr(q_st_2[s, t] >= sum(x_prev[s, l, t, v] for l in teachers for v in V[s]) - 
                         sum(x_2[s, l, t, v] for l in teachers for v in V[s]))
for s in students:
    model_soft2.addConstr(Q_maxtime_2 >= sum(q_st_2[s, t] for t in timeslots))

#Assignment Q_maxsubject
for s in students:
    for v in V[s]:
        for t in timeslots:
            model_soft2.addConstr(q_stv_2[s, t, v] >= sum(x_2[s, l, t, v] for l in teachers) - sum(x_prev[s, l, t, v] for l in teachers))

for s in students:
    for v in V[s]:
        for t in timeslots:
            model_soft2.addConstr(q_stv_2[s, t, v] >= sum(x_prev[s, l, t, v] for l in teachers) - sum(x_2[s, l, t, v] for l in teachers))
for s in students:
    model_soft2.addConstr(Q_maxsubject_2 >= sum(q_stv_2[s, t, v] for v in V[s] for t in timeslots))

#Assignment Q_maxteacher
for s in students:
    for l in teachers:
        for v in courses:
            for t in timeslots:
                model_soft2.addConstr(y_slv_prev_2[s,l,v] >= x_prev[s,l,t,v])
for s in students:
    for l in teachers:
        for v in courses:
            model_soft2.addConstr(y_slv_prev_2[s,l,v] <= sum(x_prev[s,l,t,v] for t in timeslots))
                
for s in students:
    for l in teachers:
        for v in courses:
            model_soft2.addConstr(q_slv_2[s,l,v] >= y_slv_2[s,l,v] - y_slv_prev_2[s,l,v])

for s in students:
    for l in teachers:
        for v in courses:
            model_soft2.addConstr(q_slv_2[s,l,v] >= y_slv_prev_2[s,l,v] - y_slv_2[s,l,v])

for s in students:
    model_soft2.addConstr(Q_maxteacher_2 >= sum(q_slv_2[s,l,v] for l in teachers for v in V[s]))



In [5]:


model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution with HARD one-on-one constraint found!")
    for s in students:
        for l in teachers:
            for t in timeslots:
                for v in V[s]:
                    if x[s, l, t, v].X > 0.5:
                        print(f"Student {s} takes {v} with {l} at timeslot {t}")


else:
    print("⚠️ No feasible solution with HARD constraints. Trying soft one-on-one constraint...")

    model_soft.optimize()
    
    if model_soft.status == GRB.OPTIMAL:
        print("✅ Optimal solution with SOFT one-on-one constraint found!")
        for s in students:
            for l in teachers:
                for t in timeslots:
                    for v in V[s]:
                        if x_1[s, l, t, v].X > 0.5:
                            print(f"Student {s} takes {v} with {l} at timeslot {t}")
                            
        print("\n⚠️ Conflicts for students with a[s] = 1 (one-on-one required):")
        for s in students:
            if a[s] == 1:
                for l in teachers:
                    for t in timeslots:
                        # Check if student s has a lesson with teacher l at time t (regardless of subject)
                        assigned = any(x[s, l, t, v].X > 0.5 for v in V[s])
                        if assigned:
                            # Look for other students who also have a lesson with this teacher at the same time
                            other_students = [
                                s2 for s2 in students if s2 != s and any(v2 in V[s2] and x[s2, l, t, v2].X > 0.5 for v2 in V[s2])
                            ]
                            if other_students:
                                print(f"❗ Student {s} has class with {l} at time {t}, but {l} also teaches to: {', '.join(other_students)}")
        
        print("Let's see if relaxing the single-teacher-per-course constraint gives a better alternative solution...")
    else:
        print("❌ No feasible solution with soft one-on-one constraint. Trying with relaxed single-teacher-per-course constraint...")
    
    model_soft2.optimize()

    if model_soft2.status == GRB.OPTIMAL:
        print("✅ Optimal solution with SOFT single-teacher-per-course constraint found!")
        for s in students:
            for l in teachers:
                for t in timeslots:
                    for v in V[s]:
                        if x_2[s, l, t, v].X > 0.5:
                            print(f"Student {s} takes {v} with {l} at timeslot {t}")

        print("\n⚠️ Conflicts: Students assigned to more than one teacher for the same course:")
        for s in students:
            for v in V[s]:
                assigned_teachers = [l for l in teachers if y_slv_2[s, l, v].x > 0.5]
                if len(assigned_teachers) > 1:
                    print(f"❗ Student {s} has course {v} with multiple teachers: {', '.join(assigned_teachers)}")
    else: 
        print("❌ No feasible solution with soft single-teacher-per-course constraint found!")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\n Total runtime: {elapsed_time:.2f} seconds")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[rosetta2] - Darwin 24.3.0 24D81)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2636099 - for non-commercial use only - registered to so___@student.kuleuven.be
Optimize a model with 2648700 rows, 1163271 columns and 4082364 nonzeros
Model fingerprint: 0xb27d2d0b
Variable types: 3 continuous, 1163268 integer (1163268 binary)
Coefficient statistics:
  Matrix range     [4e-02, 2e+00]
  Objective range  [2e-04, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 2638782 rows and 1158203 columns
Presolve time: 0.87s
Presolved: 9918 rows, 5068 columns, 104454 nonzeros
Variable types: 0 continuous, 5068 integer (5065 binary)
Performing another presolve...
Presolve removed 7127 rows and 978 columns
Presolve time: 0.21s

Root relaxation: objective -2.389422e-01, 2784 iterations, 0.13 seconds (0.12 work units)

    Nodes    |    Curre

In [8]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment
from openpyxl.utils import get_column_letter



# === Time Mapping ===
def get_day_block_and_time(t):
    if 1 <= t <= 6:
        return "monday", "morning" if t <= 3 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 1]
    elif 7 <= t <= 12:
        return "tuesday", "morning" if t <= 9 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 7]
    elif 13 <= t <= 15:
        return "wednesday", "all", ["09.00u", "10.00u", "11.00u"][t - 13]
    elif 16 <= t <= 21:
        return "thursday", "morning" if t <= 18 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 16]
    elif 22 <= t <= 27:
        return "friday", "morning" if t <= 24 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 22]
    else:
        return None, None, None

# === Layout
columns = ["monday", "tuesday", "wednesday", "thursday", "friday"]
block_times = {
    "morning": ["09.00u", "10.00u", "11.00u"],
    "afternoon": ["13.00u", "14.00u", "15.00u"],
    "all": ["09.00u", "10.00u", "11.00u"]  # for Wednesday
}
conflict_fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")

# === Create workbook
wb = Workbook()
wb.remove(wb.active)  # remove default sheet

for s in students:
    student_name = s
    ws = wb.create_sheet(title=student_name[:31])  # Sheet names max 31 chars

    # Build empty layout
    time_slots = sorted(set(block_times["morning"] + block_times["afternoon"]))
    ws.append([""] + [day.capitalize() for day in columns])  # Header row

    for hour in time_slots:
        ws.append([hour] + ["-" for _ in columns])
    
    if model.status == GRB.OPTIMAL: 

        # Fill entries
        for l in teachers:
            for v in V.get(s, []):
                for t in timeslots:
                    try:
                        if x[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day or not hour:
                                continue
                            col = columns.index(day) + 2  # +2 because column 1 = hour
                            row = time_slots.index(hour) + 2  # +2 because header row
                            teacher = l
                            entry = f"{v} ({teacher})"
                            cell = ws.cell(row=row, column=col)
                            if cell.value == "-" or not cell.value:
                                cell.value = entry
                            else:
                                cell.value += f" / {entry}"
                    except:
                        continue
                        
    elif model_soft.status == GRB.OPTIMAL: 

        # Fill entries
        for l in teachers:
            for v in V.get(s, []):
                for t in timeslots:
                    try:
                        if x_1[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day or not hour:
                                continue
                            col = columns.index(day) + 2  # +2 because column 1 = hour
                            row = time_slots.index(hour) + 2  # +2 because header row
                            teacher = l
                            entry = f"{v} ({teacher})"
                            cell = ws.cell(row=row, column=col)
                            if cell.value == "-" or not cell.value:
                                cell.value = entry
                            else:
                                cell.value += f" / {entry}"
                    except:
                        continue
    elif model_soft2.status == GRB.OPTIMAL: 

        # Fill entries
        for l in teachers:
            for v in V.get(s, []):
                for t in timeslots:
                    try:
                        if x_2[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day or not hour:
                                continue
                            col = columns.index(day) + 2  # +2 because column 1 = hour
                            row = time_slots.index(hour) + 2  # +2 because header row
                            teacher = l
                            entry = f"{v} ({teacher})"
                            cell = ws.cell(row=row, column=col)
                            if cell.value == "-" or not cell.value:
                                cell.value = entry
                            else:
                                cell.value += f" / {entry}"
                    except:
                        continue

    # Format all cells nicely
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=2, max_col=6):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical='center', horizontal='center')
    for col in range(1, 7):
        ws.column_dimensions[get_column_letter(col)].width = 20

# === Save
wb.save("student_schedule_per_sheet_anonymous.xlsx")


In [10]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment
from openpyxl.utils import get_column_letter

# === Conflict color for changed teacher
conflict_fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")

# NEW: for one-on-one violations (model_soft)
one_on_one_violation_fill = PatternFill(start_color="FFF59D", end_color="FFF59D", fill_type="solid")  # Yellow

# NEW: for multiple teachers (model_soft2)
multi_teacher_violation_fill = PatternFill(start_color="90CAF9", end_color="90CAF9", fill_type="solid")  # Light blue

# === Time mapping
def get_day_block_and_time(t):
    if 1 <= t <= 6:
        return "monday", "morning" if t <= 3 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 1]
    elif 7 <= t <= 12:
        return "tuesday", "morning" if t <= 9 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 7]
    elif 13 <= t <= 15:
        return "wednesday", "all", ["09.00u", "10.00u", "11.00u"][t - 13]
    elif 16 <= t <= 21:
        return "thursday", "morning" if t <= 18 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 16]
    elif 22 <= t <= 27:
        return "friday", "morning" if t <= 24 else "afternoon", ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 22]
    else:
        return None, None, None

# === Layout
columns = ["monday", "tuesday", "wednesday", "thursday", "friday"]
block_times = {
    "morning": ["09.00u", "10.00u", "11.00u"],
    "afternoon": ["13.00u", "14.00u", "15.00u"],
    "all": ["09.00u", "10.00u", "11.00u"]
}

if model.status == GRB.OPTIMAL: 
    # === Precompute changes in teacher per (student, course)
    teacher_changes = {}
    for s in students:
        for v in V[s]:
            curr_teachers = [l for l in teachers if y_slv[s, l, v].X > 0.5]
            prev_teachers = [l for l in teachers if y_slv_prev[s, l, v].X > 0.5]
            if v == "Zelfstudie" or not prev_teachers:
                continue
            for l_new in curr_teachers:
                if l_new not in prev_teachers:
                    teacher_changes[(s, v)] = (l_new, prev_teachers)
    # === Create Excel workbook
    wb = Workbook()
    wb.remove(wb.active)

    for s in students:
        student_name = s
        ws = wb.create_sheet(title=student_name[:31])

        # Build empty grid layout
        time_slots = sorted(set(block_times["morning"] + block_times["afternoon"]))
        ws.append([""] + [day.capitalize() for day in columns])

        for hour in time_slots:
            ws.append([hour] + ["-" for _ in columns])

           # Fill entries for student
        for l in teachers:
            for v in V.get(s, []):
                for t in timeslots:
                    try:
                        if x[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day or not hour:
                                continue
                            col = columns.index(day) + 2
                            row = time_slots.index(hour) + 2

                            teacher_initial = l

                            # Check for teacher change
                            highlight = False
                            entry = f"{v} ({teacher_initial})"

                            # Check for teacher change
                            if (s, v) in teacher_changes:
                                new_l, old_ls = teacher_changes[(s, v)]
                                if l == new_l:
                                    old_initials = "/".join(ol for ol in old_ls)
                                    new_initial = new_l
                                    entry = f"{v} ({old_initials} → {new_initial})"
                                    highlight = True

                            cell = ws.cell(row=row, column=col)
                            if cell.value == "-" or not cell.value:
                                cell.value = entry
                            else:
                                cell.value += f" / {entry}"

                            if highlight:
                                cell.fill = conflict_fill
                    except:
                        continue

elif model_soft.status == GRB.OPTIMAL: 
    # === Precompute changes in teacher per (student, course)
    teacher_changes = {}
    for s in students:
        for v in V[s]:
            curr_teachers = [l for l in teachers if y_slv_1[s, l, v].X > 0.5]
            prev_teachers = [l for l in teachers if y_slv_prev_1[s, l, v].X > 0.5]
            if v == "Zelfstudie" or not prev_teachers:
                continue
            for l_new in curr_teachers:
                if l_new not in prev_teachers:
                    teacher_changes[(s, v)] = (l_new, prev_teachers)
    # === Create Excel workbook
    wb = Workbook()
    wb.remove(wb.active)

    for s in students:
        student_name = s
        ws = wb.create_sheet(title=student_name[:31])

        # Build empty grid layout
        time_slots = sorted(set(block_times["morning"] + block_times["afternoon"]))
        ws.append([""] + [day.capitalize() for day in columns])

        for hour in time_slots:
            ws.append([hour] + ["-" for _ in columns])

           # Fill entries for student
        for l in teachers:
            for v in V.get(s, []):
                for t in timeslots:
                    try:
                        if x_1[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day or not hour:
                                continue
                            col = columns.index(day) + 2
                            row = time_slots.index(hour) + 2

                            teacher_initial = l

                            # Check for teacher change
                            highlight = False
                            entry = f"{v} ({teacher_initial})"

                            # Check for teacher change
                            if (s, v) in teacher_changes:
                                new_l, old_ls = teacher_changes[(s, v)]
                                if l == new_l:
                                    old_initials = "/".join(ol for ol in old_ls)
                                    new_initial = new_l
                                    entry = f"{v} ({old_initials} → {new_initial})"
                                    highlight = True
                                    
                            # Additional conflict: student a[s]==1 but shared teacher at time t
                            if a[s] == 1:
                                # Check how many other students share teacher `l` at time `t`
                                shared = False
                                for s2 in students:
                                    if s2 != s and any(x_1[s2, l, t, v2].X > 0.5 for v2 in V.get(s2, [])):
                                        shared = True
                                        break
                                if shared:
                                    cell.fill = one_on_one_violation_fill

                            cell = ws.cell(row=row, column=col)
                            if cell.value == "-" or not cell.value:
                                cell.value = entry
                            else:
                                cell.value += f" / {entry}"

                            if highlight:
                                cell.fill = conflict_fill
                    except:
                        continue
                        
elif model_soft2.status == GRB.OPTIMAL: 
    # === Precompute changes in teacher per (student, course)
    teacher_changes = {}
    for s in students:
        for v in V[s]:
            curr_teachers = [l for l in teachers if y_slv_2[s, l, v].X > 0.5]
            prev_teachers = [l for l in teachers if y_slv_prev_2[s, l, v].X > 0.5]
            if v == "Zelfstudie" or not prev_teachers:
                continue
            for l_new in curr_teachers:
                if l_new not in prev_teachers:
                    teacher_changes[(s, v)] = (l_new, prev_teachers)
    # === Create Excel workbook
    wb = Workbook()
    wb.remove(wb.active)

    for s in students:
        student_name = s
        ws = wb.create_sheet(title=student_name[:31])

        # Build empty grid layout
        time_slots = sorted(set(block_times["morning"] + block_times["afternoon"]))
        ws.append([""] + [day.capitalize() for day in columns])

        for hour in time_slots:
            ws.append([hour] + ["-" for _ in columns])

           # Fill entries for student
        for l in teachers:
            for v in V.get(s, []):
                for t in timeslots:
                    try:
                        if x_2[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day or not hour:
                                continue
                            col = columns.index(day) + 2
                            row = time_slots.index(hour) + 2

                            teacher_initial = l

                            # Check for teacher change
                            highlight = False
                            entry = f"{v} ({teacher_initial})"

                            # Check for teacher change
                            if (s, v) in teacher_changes:
                                new_l, old_ls = teacher_changes[(s, v)]
                                if l == new_l:
                                    old_initials = "/".join(ol for ol in old_ls)
                                    new_initial = new_l
                                    entry = f"{v} ({old_initials} → {new_initial})"
                                    highlight = True
                                    
                            # Additional conflict: student assigned multiple teachers for the same course
                            assigned_teachers = [l2 for l2 in teachers if y_slv_2[s, l2, v].X > 0.5]
                            if len(assigned_teachers) > 1:
                                cell.fill = multi_teacher_violation_fill

                            cell = ws.cell(row=row, column=col)
                            if cell.value == "-" or not cell.value:
                                cell.value = entry
                            else:
                                cell.value += f" / {entry}"

                            if highlight:
                                cell.fill = conflict_fill
                    except:
                        continue
    

    # Format
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=2, max_col=6):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical='center', horizontal='center')
    for col in range(1, 7):
        ws.column_dimensions[get_column_letter(col)].width = 22

# === Save
wb.save("student_schedule_per_sheet_highlighted_anonymous.xlsx")


In [14]:
import pandas as pd

# === Time mapping stays unchanged ===
def get_day_block_and_time(t):
    if 1 <= t <= 6:
        day = "monday"
        block = "morning" if t <= 3 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 1]
    elif 7 <= t <= 12:
        day = "tuesday"
        block = "morning" if t <= 9 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 7]
    elif 13 <= t <= 15:
        day = "wednesday"
        block = "all"
        hour = ["09.00u", "10.00u", "11.00u"][t - 13]
    elif 16 <= t <= 21:
        day = "thursday"
        block = "morning" if t <= 18 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 16]
    elif 22 <= t <= 27:
        day = "friday"
        block = "morning" if t <= 24 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 22]
    else:
        return None, None, None
    return day, block, hour

# === Columns for schedule layout ===
columns = [
    "monday_morning", "monday_afternoon",
    "tuesday_morning", "tuesday_afternoon",
    "wednesday",
    "thursday_morning", "thursday_afternoon",
    "friday_morning", "friday_afternoon"
]

# === Build data rows using s1, l1, etc. ===
data = []

if model.status == GRB.OPTIMAL: 
    for s in students:
        schedule = {col: "-" for col in columns}
        student_courses = V.get(s, [])

        for l in teachers:
            for v in student_courses:
                for t in timeslots:
                    try:
                        if x[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day:
                                continue
                            col = f"{day}_{block}" if block != "all" else "wednesday"
                            entry = f"{hour} {v} ({l})"
                            if schedule[col] == "-":
                                schedule[col] = entry
                            else:
                                schedule[col] += f" / {entry}"
                    except:
                        continue

        data.append({"student": s, **schedule})

elif model_soft.status == GRB.OPTIMAL: 
    for s in students:
        schedule = {col: "-" for col in columns}
        student_courses = V.get(s, [])

        for l in teachers:
            for v in student_courses:
                for t in timeslots:
                    try:
                        if x_1[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day:
                                continue
                            col = f"{day}_{block}" if block != "all" else "wednesday"
                            entry = f"{hour} {v} ({l})"
                            if schedule[col] == "-":
                                schedule[col] = entry
                            else:
                                schedule[col] += f" / {entry}"
                    except:
                        continue

        data.append({"student": s, **schedule})
        
elif model_soft2.status == GRB.OPTIMAL: 
    for s in students:
        schedule = {col: "-" for col in columns}
        student_courses = V.get(s, [])

        for l in teachers:
            for v in student_courses:
                for t in timeslots:
                    try:
                        if x_2[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day:
                                continue
                            col = f"{day}_{block}" if block != "all" else "wednesday"
                            entry = f"{hour} {v} ({l})"
                            if schedule[col] == "-":
                                schedule[col] = entry
                            else:
                                schedule[col] += f" / {entry}"
                    except:
                        continue

        data.append({"student": s, **schedule})

# === Save to Excel ===
df = pd.DataFrame(data)
df = df[["student"] + columns]
df.to_excel("student_schedule_together_anonymous.xlsx", index=False)


In [15]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill

# === Time mapping function ===
def get_day_block_and_time(t):
    if 1 <= t <= 6:
        day = "monday"
        block = "morning" if t <= 3 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 1]
    elif 7 <= t <= 12:
        day = "tuesday"
        block = "morning" if t <= 9 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 7]
    elif 13 <= t <= 15:
        day = "wednesday"
        block = "all"
        hour = ["09.00u", "10.00u", "11.00u"][t - 13]
    elif 16 <= t <= 21:
        day = "thursday"
        block = "morning" if t <= 18 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 16]
    elif 22 <= t <= 27:
        day = "friday"
        block = "morning" if t <= 24 else "afternoon"
        hour = ["09.00u", "10.00u", "11.00u", "13.00u", "14.00u", "15.00u"][t - 22]
    else:
        return None, None, None
    return day, block, hour

# === Column layout ===
columns = [
    "monday_morning", "monday_afternoon",
    "tuesday_morning", "tuesday_afternoon",
    "wednesday",
    "thursday_morning", "thursday_afternoon",
    "friday_morning", "friday_afternoon"
]

if model.status == GRB.OPTIMAL: 
    # === Step 1: Identify changed (s,v) assignments ===
    changed_teachers = set()
    for s in students:
        for v in V[s]:
            current_l = next((l for l in teachers if y_slv[s, l, v].X > 0.5), None)
            prev_l = next((l for l in teachers if y_slv_prev[s, l, v].X > 0.5), None)
            if current_l and prev_l and current_l != prev_l:
                changed_teachers.add((s, v))

    # === Step 2: Build schedule ===
    data = []
    highlight_cells = set()  # to track cells to highlight later

    for s in students:
        schedule = {col: "-" for col in columns}
        student_courses = V.get(s, [])

        for l in teachers:
            for v in student_courses:
                for t in timeslots:
                    try:
                        if x[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day:
                                continue
                            col = f"{day}_{block}" if block != "all" else "wednesday"
                            entry = f"{hour} {v} ({l})"
                            if schedule[col] == "-":
                                schedule[col] = entry
                            else:
                                schedule[col] += f" / {entry}"
                            if (s, v) in changed_teachers:
                                highlight_cells.add((s, col))
                    except:
                        continue

        data.append({"student": s, **schedule})
        
elif model_soft.status == GRB.OPTIMAL: 
    # === Step 1: Identify changed (s,v) assignments ===
    changed_teachers = set()
    for s in students:
        for v in V[s]:
            current_l = next((l for l in teachers if y_slv_1[s, l, v].X > 0.5), None)
            prev_l = next((l for l in teachers if y_slv_prev_1[s, l, v].X > 0.5), None)
            if current_l and prev_l and current_l != prev_l:
                changed_teachers.add((s, v))

    # === Step 2: Build schedule ===
    data = []
    highlight_cells = set()  # to track cells to highlight later

    for s in students:
        schedule = {col: "-" for col in columns}
        student_courses = V.get(s, [])

        for l in teachers:
            for v in student_courses:
                for t in timeslots:
                    try:
                        if x_1[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day:
                                continue
                            col = f"{day}_{block}" if block != "all" else "wednesday"
                            entry = f"{hour} {v} ({l})"
                            if schedule[col] == "-":
                                schedule[col] = entry
                            else:
                                schedule[col] += f" / {entry}"
                            if (s, v) in changed_teachers:
                                highlight_cells.add((s, col))
                            # Additional conflict: student a[s]==1 but shared teacher at time t
                            if a[s] == 1:
                                # Check how many other students share teacher `l` at time `t`
                                shared = False
                                for s2 in students:
                                    if s2 != s and any(x_1[s2, l, t, v2].X > 0.5 for v2 in V.get(s2, [])):
                                        shared = True
                                        break
                                if shared:
                                    cell.fill = one_on_one_violation_fill
                    except:
                        continue

        data.append({"student": s, **schedule})

elif model_soft2.status == GRB.OPTIMAL: 
    # === Step 1: Identify changed (s,v) assignments ===
    changed_teachers = set()
    for s in students:
        for v in V[s]:
            current_l = next((l for l in teachers if y_slv_2[s, l, v].X > 0.5), None)
            prev_l = next((l for l in teachers if y_slv_prev_2[s, l, v].X > 0.5), None)
            if current_l and prev_l and current_l != prev_l:
                changed_teachers.add((s, v))

    # === Step 2: Build schedule ===
    data = []
    highlight_cells = set()  # to track cells to highlight later

    for s in students:
        schedule = {col: "-" for col in columns}
        student_courses = V.get(s, [])

        for l in teachers:
            for v in student_courses:
                for t in timeslots:
                    try:
                        if x_2[s, l, t, v].X > 0.5:
                            day, block, hour = get_day_block_and_time(t)
                            if not day:
                                continue
                            col = f"{day}_{block}" if block != "all" else "wednesday"
                            entry = f"{hour} {v} ({l})"
                            if schedule[col] == "-":
                                schedule[col] = entry
                            else:
                                schedule[col] += f" / {entry}"
                            if (s, v) in changed_teachers:
                                highlight_cells.add((s, col))
                                
                            # Additional conflict: student assigned multiple teachers for the same course
                            assigned_teachers = [l2 for l2 in teachers if y_slv_2[s, l2, v].X > 0.5]
                            if len(assigned_teachers) > 1:
                                cell.fill = multi_teacher_violation_fill

                    except:
                        continue

        data.append({"student": s, **schedule})

# === Step 3: Save to Excel ===
df = pd.DataFrame(data)
df = df[["student"] + columns]
file_path = "student_schedule_together_highlighted_anonymous.xlsx"
df.to_excel(file_path, index=False)

# === Step 4: Highlight cells where teacher changed ===
wb = openpyxl.load_workbook(file_path)
ws = wb.active
highlight_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Map student ID to row
student_to_row = {df.iloc[i]["student"]: i+2 for i in range(len(df))}  # +2 because Excel rows start at 1 and first row is header

# Apply highlighting
for s, col in highlight_cells:
    row = student_to_row[s]
    col_idx = df.columns.get_loc(col) + 1  # +1 because Excel columns start at 1
    ws.cell(row=row, column=col_idx).fill = highlight_fill

# Save final file
wb.save(file_path)
print("Saved:", file_path)


Saved: student_schedule_together_highlighted_anonymous.xlsx
